In [14]:
#import libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from Data_Prep import *

In [15]:
# Defining the file path for CAN datasets
in_can = "/media/arupreza/Assets/Data/Dataset_Yoonji/CAN/"

In [16]:
# List all files in the directory and store them in a variable
def get_file_list(directory):
    try:
        files = sorted(os.listdir(directory))  # Sort the files for consistent indexing
        print(f"Files in '{directory}':")
        for i, file in enumerate(files):
            print(f"{i + 1}: {file}")  # Display index for each file
        return files
    except FileNotFoundError:
        print(f"The directory '{directory}' does not exist.")
        return []
    except PermissionError:
        print(f"Permission denied for accessing the directory '{directory}'.")
        return []


In [17]:
# Main workflow
file_list = get_file_list(in_can)  # Get all filenames

# Prepend the directory path to the file name
file_path = os.path.join(in_can, file_list[0])  # Full file path

Files in '/media/arupreza/Assets/Data/Dataset_Yoonji/CAN/':
1: normal_CAN.trc
2: normal_CAN_30(1).trc
3: normal_CAN_30(10).trc
4: normal_CAN_30(2).trc
5: normal_CAN_30(3).trc
6: normal_CAN_30(4).trc
7: normal_CAN_30(5).trc
8: normal_CAN_30(6).trc
9: normal_CAN_30(7).trc
10: normal_CAN_30(8).trc
11: normal_CAN_30(9).trc
12: replay_attack1_CAN_60(1).trc
13: replay_attack1_CAN_60(2).trc
14: replay_attack1_CAN_60(3).trc
15: replay_attack1_CAN_60(4).trc
16: replay_attack1_CAN_60(5).trc
17: replay_attack2_CAN_60(1).trc
18: replay_attack2_CAN_60(2).trc
19: replay_attack2_CAN_60(3).trc
20: replay_attack2_CAN_60(4).trc
21: replay_attack2_CAN_60(5).trc


In [18]:
a = Convert_to_df(file_path)

NameError: name 'pd' is not defined

In [57]:
a

,No,Time_Offset,Type,CAN_ID,Data_Length,One,Two,Three,Four,Five,Six,Seven,Eight
17,1),5.6,Rx,043F,8,01,45,60,FF,66,00,00,00
18,2),5.9,Rx,0440,8,FF,91,00,00,FF,00,00,00
19,3),6.3,Rx,0316,8,05,20,02,0A,20,13,00,7F
20,4),6.5,Rx,0260,8,05,20,00,30,3C,8D,5F,0E
21,5),6.8,Rx,018F,8,00,2B,20,00,00,3F,00,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
634738,634722),304261.5,Rx,043F,8,07,47,60,FF,6B,00,00,00
634739,634723),304261.7,Rx,0370,8,FF,20,E0,80,FF,00,00,C0
634740,634724),304262.0,Rx,0440,8,FF,4E,00,00,FF,00,00,00
634741,634725),304262.2,Rx,04B0,8,00,00,00,80,00,40,00,80
